In [1]:
from azureml.core.webservice import AciWebservice, Webservice
from azureml.core.image import ContainerImage
from azureml.core.experiment import Experiment
from azureml.core.workspace import Workspace
from azureml.core.environment import Environment
from azureml.core.conda_dependencies import CondaDependencies
from azureml.core.model import InferenceConfig,Model
import sys

In [2]:
ws = Workspace.from_config()

In [3]:
webservice_name = 'turbofan-rul'

run = [x for x in ws.experiments['gbr-turbofan'].get_runs()][0]
run_metrics = run.get_metrics()

print(run)

model = run.register_model(model_name = "turbofan-rul", model_path = "outputs/model.pkl", tags={"mae":run_metrics["mae"],"python version": sys.version[0:6]})

deployment_config = AciWebservice.deploy_configuration(cpu_cores = 1, memory_gb = 1,auth_enabled=True)

Run(Experiment: gbr-turbofan,
Id: 7e07b113-d164-42c4-ab49-e156f7220410,
Type: None,
Status: Completed)


In [8]:
myenv = Environment.get(workspace=ws,name="sklearn 20.2")
myenv.inferencing_stack_version='latest'

inference_config = InferenceConfig(entry_script="score.py",
                                   environment=myenv)
                               
try:
    service = Webservice(ws, webservice_name)        
    service.delete()    
except Exception as e:
    print("No Existing Service")

service = Model.deploy(ws, webservice_name, [model], inference_config, deployment_config)

#service.get_logs()
service.wait_for_deployment(show_output = True)

#print(service.get_keys())
print(service.scoring_uri)

Tips: You can try get_logs(): https://aka.ms/debugimage#dockerlog or local deployment: https://aka.ms/debugimage#debug-locally to debug if deployment takes longer than 10 minutes.
Running
2021-09-27 01:06:18+00:00 Creating Container Registry if not exists.
2021-09-27 01:06:18+00:00 Registering the environment.
2021-09-27 01:06:20+00:00 Building image..
2021-09-27 01:16:50+00:00 Generating deployment configuration.
2021-09-27 01:16:51+00:00 Submitting deployment to compute.
2021-09-27 01:16:54+00:00 Checking the status of deployment turbofan-rul..
2021-09-27 01:25:01+00:00 Checking the status of inference endpoint turbofan-rul.
Failed


Service deployment polling reached non-successful terminal state, current service state: Failed
Operation ID: 67611988-7d52-44a6-b847-836c2b421987
More information can be found using '.get_logs()'
Error:
{
  "code": "AciDeploymentFailed",
  "statusCode": 400,
  "message": "Aci Deployment failed with exception: Error in entry script, ModuleNotFoundError: No module named 'sklearn.ensemble._gb', please run print(service.get_logs()) to get details.",
  "details": [
    {
      "code": "CrashLoopBackOff",
      "message": "Error in entry script, ModuleNotFoundError: No module named 'sklearn.ensemble._gb', please run print(service.get_logs()) to get details."
    }
  ]
}



WebserviceException: WebserviceException:
	Message: Service deployment polling reached non-successful terminal state, current service state: Failed
Operation ID: 67611988-7d52-44a6-b847-836c2b421987
More information can be found using '.get_logs()'
Error:
{
  "code": "AciDeploymentFailed",
  "statusCode": 400,
  "message": "Aci Deployment failed with exception: Error in entry script, ModuleNotFoundError: No module named 'sklearn.ensemble._gb', please run print(service.get_logs()) to get details.",
  "details": [
    {
      "code": "CrashLoopBackOff",
      "message": "Error in entry script, ModuleNotFoundError: No module named 'sklearn.ensemble._gb', please run print(service.get_logs()) to get details."
    }
  ]
}
	InnerException None
	ErrorResponse 
{
    "error": {
        "message": "Service deployment polling reached non-successful terminal state, current service state: Failed\nOperation ID: 67611988-7d52-44a6-b847-836c2b421987\nMore information can be found using '.get_logs()'\n